In [2]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)
drive.mount('/content/drive')

Mounted at /gdrive
Mounted at /content/drive


# Conv1D + RNN (GRU)
- 1D컨브넷이 입력 패치를 독립적으로 처리하기 때문에 <u>타임스텝의 순서에는 민감하지 않음</u>
- 컨브넷의 한계
  - 장기간 패턴을 인식하기 위해 많은 합성곱층과 풀링층을 쌓을 수 있음 ➡️ 상위층은 원본 입력에서 긴 범위를 보게되며 순서를 감지하기에 부족함 
  - 온도 예측 문제에 1D를 적용하면 정확성이 떨어짐 (패턴의 시간축의 위치를 고려하지 않았기 때문)

In [4]:
weather_dir = '/content/drive/MyDrive/Data/Encoding/jena_climate_2009_2016.csv'

In [5]:
# 데이터 불러오기 
import os

fname = os.path.join(weather_dir)

f = open(fname)
data = f.read()
f.close()

In [7]:
# 데이터 탐색 
# 총 42만 551줄 
# header는 14개의 날씨 정보를 포함하고 있음 

lines = data.split('\n')
header = lines[0].split(',')
lines = lines[1:]

print(len(lines))
print(header)

420451
['"Date Time"', '"p (mbar)"', '"T (degC)"', '"Tpot (K)"', '"Tdew (degC)"', '"rh (%)"', '"VPmax (mbar)"', '"VPact (mbar)"', '"VPdef (mbar)"', '"sh (g/kg)"', '"H2OC (mmol/mol)"', '"rho (g/m**3)"', '"wv (m/s)"', '"max. wv (m/s)"', '"wd (deg)"']


In [8]:
# Parsing the data 

import numpy as np

float_data = np.zeros((len(lines), len(header)-1))

for i, line in enumerate(lines):
  values = [float(x) for x in line.split(',')[1:]]
  float_data[i, :] = values

In [9]:
# 데이터 전처리 1 : 데이터의 범위가 다름, 정규화 하여 비슷한 범위를 가진 작은 값으로 변경 
# 시계열 특성에 대해 평균을 빼고 표준 편차로 나눠 전처리 

mean = float_data[:200000].mean()
float_data -= mean

std = float_data[:200000].std(axis=0)
float_data /= std

In [10]:
# 데이터 전처리 2: float data 배열을 받아 과거 데이터의 배치와 미래 타깃 온도를 추출하는 파이썬 제너레이터 생성 

def generator(data, lookback, delay, min_index, max_index,
              shuffle=False, batch_size=128, step=6):
  if max_index is None:
    max_index = len(data) - delay - 1
  i = min_index + lookback
  while 1:
    if shuffle:
      rows = np.random.randint(min_index + lookback, max_index, size=batch_size)
    else:
      if i + batch_size >= max_index:
        i = min_index + lookback
      rows = np.arange(i, min(i + batch_size, max_index))
      i += len(rows)
    
    samples = np.zeros((len(rows), lookback // step, data.shape[-1]))
    targets = np.zeros((len(rows),))
    for j, row in enumerate(rows):
      indices = range(rows[j] - lookback, rows[j], step)
      samples[j] = data[indices]
      targets[j] = data[rows[j]+delay][1]
    yield samples, targets

In [11]:
step =3 
lookback = 720
delay = 144

train_gen = generator(float_data,
                      lookback = lookback,
                      delay = delay,
                      min_index=0,
                      max_index=200000,
                      shuffle=True,
                      step=step)

val_gen = generator(float_data,
                    lookback = lookback,
                    delay = delay,
                    min_index=200001,
                    max_index=300000,
                    shuffle=True,
                    step=step)

test_gen = generator(float_data,
                    lookback = lookback,
                    delay = delay,
                    min_index=300001,
                    max_index=None,
                    shuffle=True,
                    step=step)

val_steps = (300000 - 200001 - lookback) //128
test_steps = (len(float_data) - 300001 - lookback) //128

### Combining CNNs and RNNs to process long sequences 
- 컨브넷의 속도와 경량함을 RNN의 순서 감지 능력과 결합 
  - 컨브넷 1D를 RNN이전에 전처리 단계로 사용 (너무 긴 시퀀스를 처리)🧹
  - Conv1D 다운샘플된 시퀀스로 변환 ➡️ 추출된 특성의 시퀀스는 RNN의 입력이됨 

#### 온도 예측 문제에 적용 🌡
- 훨씬 긴 시퀀스를 다룰 수 있으므로, 더 오래전 데이터를 바라보거나 시계열 데이터를 더 정밀히 볼 수 있음 
- steps를 절반으로 줄임 : 온도 데이터가 30분마다 1포인트씩 샘플링 되므로 결과 시계열 데이터는 2배로 길어짐 

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop

model = Sequential()
model.add(layers.Conv1D(32, 5, activation='relu', input_shape=(None, float_data.shape[-1])))
model.add(layers.MaxPooling1D(3))
model.add(layers.Conv1D(32, 5, activation='relu'))
model.add(layers.GRU(32, dropout=0.1, recurrent_dropout=0.5))
model.add(layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, None, 32)          2272      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, None, 32)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 32)          5152      
_________________________________________________________________
gru (GRU)                    (None, 32)                6336      
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 13,793
Trainable params: 13,793
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.compile(optimizer=RMSprop(), loss='mae')

history =model.fit_generator(train_gen,
                             steps_per_epoch=500,
                             epochs=20,
                             validation_data=val_gen,
                             validation_steps= val_steps)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
500/500 [==============================] - 265s 465ms/step - loss: 7.8227 - val_loss: 0.7667
Epoch 2/20
500/500 [==============================] - 232s 463ms/step - loss: 0.8065 - val_loss: 0.7351
Epoch 3/20
500/500 [==============================] - 231s 462ms/step - loss: 0.6949 - val_loss: 0.4781
Epoch 4/20
500/500 [==============================] - 229s 458ms/step - loss: 0.6058 - val_loss: 0.5888
Epoch 5/20
500/500 [==============================] - 229s 458ms/step - loss: 0.5391 - val_loss: 0.5882
Epoch 6/20
500/500 [==============================] - 227s 455ms/step - loss: 0.4980 - val_loss: 0.4050
Epoch 7/20
500/500 [==============================] - 229s 458ms/step - loss: 0.4703 - val_loss: 0.4781
Epoch 8/20
500/500 [==============================] - 233s 466ms/step - loss: 0.4553 - val_loss: 0.4833
Epoch 9/20
500/500 [==============================] - 231s 462ms/step - loss: 0.4383 - val_loss: 0.3650
Epoch 10/20
500/500 [==============================] - 229s 459m

✅ 결과 : GRU모델 만큼 좋지 않으나 훨씬 빠르기 때문에 데이터를 2배 더 많이 처리할 수 있음 